In [2]:
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import pandas as pd
import io
import zipfile
import re

In [3]:
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
print("Setting up Chrome options")

PATH = "C:\\Program Files (x86)\\chromedriver.exe"
service = Service(PATH)

print("Initializing WebDriver")
driver = webdriver.Chrome(service=service, options=chrome_options)
csv_files = []

# Function to perform human-like cursor movements
def human_like_hover(driver, element, pause_time=1):
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    time.sleep(pause_time)

print("Opening URL")
driver.get("https://s3.amazonaws.com/tripdata/index.html")

# Wait for the table body to load
try:
    print("Waiting for table body to load")
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "tbody-content"))
    )
    print("Table body loaded successfully")
except Exception as e:
    print(f"Timeout waiting for page to load: {e}")
    driver.quit()
    exit()

# Verify the content of the table body
try:
    print("Finding table body element")
    tbody = driver.find_element(By.ID, "tbody-content")
    print("Table body inner HTML:")
    print(tbody.get_attribute('innerHTML'))
except Exception as e:
    print(f"Error finding elements: {e}")
    driver.quit()
    exit()

# Wait for the links to be present
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, "a"))
    )
    print("Links are present in the table body")
except Exception as e:
    print(f"Timeout waiting for links: {e}")
    driver.quit()
    exit()

# Find all anchor tags within the table body
try:
    print("Finding all anchor tags within the table body")
    links = tbody.find_elements(By.TAG_NAME, "a")
except Exception as e:
    print(f"Error finding elements: {e}")
    driver.quit()
    exit()

print(f"Found {len(links)} links")

# Move cursor over each link and print the href
for link in links:
    try:
        href = link.get_attribute("href")
        if href:
            # Move the cursor to the link
            print(f"Hovering over link: {href}")
            human_like_hover(driver, link, pause_time=random.uniform(0.1, 0.3))
            print(f"Link: {href}")
            if "zip" in href:
                csv_files.append(href)
        # Random delay to mimic human behavior
        time.sleep(random.uniform(0.5, 1))
    except Exception as e:
        print(f"Error processing link: {e}")

print("Closing driver")
driver.quit()

print(f"Found {len(csv_files)} urls files:")

Initializing WebDriver
Opening URL
Waiting for table body to load
Table body loaded successfully
Finding table body element
Table body inner HTML:

        
Links are present in the table body
Finding all anchor tags within the table body
Found 125 links
Hovering over link: https://s3.amazonaws.com/tripdata/2013-citibike-tripdata.zip
Link: https://s3.amazonaws.com/tripdata/2013-citibike-tripdata.zip
Error processing link: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.182)
Stacktrace:
	GetHandleVerifier [0x00007FF690F9EEA2+31554]
	(No symbol) [0x00007FF690F17ED9]
	(No symbol) [0x00007FF690DD872A]
	(No symbol) [0x00007FF690DAD995]
	(No symbol) [0x00007FF690E544D7]
	(No symbol) [0x00007FF690E6C051]
	(No symbol) [0x00007FF690E4CDD3]
	(No symbol) [0x00007FF690E1A33B]
	(No symbol) [0x00007FF690E1AED1]
	GetHandleVerifier [0x00007FF6912A8B1D+3217341]
	GetHandleVerifier [0x00007FF6912F5AE3+3532675]
	GetHandleVerif

KeyboardInterrupt: 

In [24]:
def download_csv(urls):
    """Download the latest version CSV files from nested ZIP archives after filtering out specific directories, and return as a list of pandas DataFrames."""
    all_data = {}
    failed_urls = []

    for url in urls:
        print(f"Downloading {url}...")
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print(f"Failed to download {url}: {e}")
            failed_urls.append(url)
            continue

        with zipfile.ZipFile(io.BytesIO(response.content)) as the_zip:
            print(f"Found {len(the_zip.namelist())} files in the ZIP archive")
            for file_name in the_zip.namelist():
                if "_MACOSX" in file_name or not file_name.endswith('.csv'):
                    continue  # Skip non-CSV and unwanted system files

                # Ignore empty or placeholder files based on a quick check of the file size if possible
                if the_zip.getinfo(file_name).file_size < 100:
                    continue  # Assumes files smaller than 100 bytes are unlikely to contain useful data

                # Try matching the original format with path
                match = re.match(r".*/(\d{6}-citibike-tripdata)(_?\d*)\.csv", file_name)
                if not match:
                    # Try matching the new format without the path
                    match = re.match(r"(\d{6}-citibike-tripdata)(_?\d*)\.csv", file_name)
                if not match:
                    # Try matching the JC format
                    match = re.match(r"JC-(\d{6}-citibike-tripdata)(_?\d*)\.csv", file_name)

                print(f"Match: {match}")
                if match:
                    print(f"Processing file: {file_name}")
                    base_id = match.group(1)
                    with the_zip.open(file_name) as file:
                        try:
                            # Define dtypes for columns based on the example rows provided
                            dtypes = {
                                'ride_id': 'float64',
                                'rideable_type': 'object',
                                'started_at': 'object',
                                'ended_at': 'object',
                                'start_station_name': 'object',
                                'start_station_id': 'float64',
                                'end_station_name': 'object',
                                'end_station_id': 'object',
                                'start_lat': 'float32',
                                'start_lng': 'float32',
                                'end_lat': 'float32',
                                'end_lng': 'float32',
                                'member_casual': 'object',
                                'tripduration': 'float32',
                                'starttime': 'object',
                                'stoptime': 'object',
                                'start_station_id': 'object',
                                'start_station_name': 'object',
                                'start_lat': 'float32',
                                'start_lng': 'float32',
                                'end_station_id': 'object',
                                'end_station_name': 'object',
                                'end_lat': 'float32',
                                'end_lng': 'float32',
                                'bikeid': 'object',
                                'usertype': 'object',
                                'birth_year': 'float32',
                                'gender': 'object'
                            }
                            temp_df = pd.read_csv(file, dtype=dtypes)
                            if temp_df.shape[0] > 0 and temp_df.shape[1] > 1:
                                if base_id in all_data:
                                    all_data[base_id] = pd.concat([all_data[base_id], temp_df], ignore_index=True)
                                else:
                                    all_data[base_id] = temp_df
                            else:
                                continue  # Skip empty data frames
                        except pd.errors.EmptyDataError:
                            continue  # Skip files with no data

    # Drop duplicates for each DataFrame in all_data
    for key in all_data:
        all_data[key] = all_data[key].drop_duplicates()

    # Print failed URLs
    if failed_urls:
        print("Failed to download the following URLs:")
        for url in failed_urls:
            print(url)

    return list(all_data.values()), failed_urls




csv_data_list, failed = download_csv(csv_files)






# Print the shape of each DataFrame to confirm data is loaded




# Print the shape of each DataFrame to confirm data is loaded
print("Data loaded successfully, DF count: ", len(csv_data_list))
print("Failed URLs: ", failed)


Found 40 files in the ZIP archive
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201309-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201309-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201311-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201311-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201307-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201307-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201308-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201308-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201306-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201306-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201310-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata

In [22]:
print("Data loaded successfully, DF count: ", len(csv_data_list))

Data loaded successfully, DF count:  132


TypeError: list indices must be integers or slices, not DataFrame